Calculate Strength of Schedule
==============================

This notebook calculates strength of schedule for both mens and womens NCAA Division 1 leagues, outputting the results to a file.

In [1]:
import pandas as pd
import numpy as np

### CONSTANTS ###

DATA_DIR = '../data/kaggle/'
YEAR     = 2025

### FUNCTIONS ###

def extract_seed_value(seed_str):
    try:
        return int(seed_str[1:])
    except:
        np.nan

def load_mens_and_womens( filename ):

    df_m_ = pd.read_csv( DATA_DIR + "M" + filename )
    df_m_["Gender"] = "M"
    
    df_w_ = pd.read_csv( DATA_DIR + "W" + filename )
    df_w_["Gender"] = "W"
    
    df_concat = pd.concat([ df_m_, df_w_ ])
    
    return df_concat

# Load Teams and Regular Season Data

Load regular season data for both mens and womens teams

In [2]:
df_reg_ = load_mens_and_womens( "RegularSeasonCompactResults.csv" )
df_reg_ = df_reg_[ df_reg_["Season"] == YEAR ]

In [3]:
df_reg_.head(5)

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,Gender
187289,2025,0,1104,110,1421,54,H,0,M
187290,2025,0,1112,93,1145,64,H,0,M
187291,2025,0,1117,80,1103,75,H,1,M
187292,2025,0,1119,67,1107,59,H,0,M
187293,2025,0,1130,69,1154,60,H,0,M


In [4]:
df_teams_ = load_mens_and_womens( "Teams.csv" )
df_teams_.info()

<class 'pandas.core.frame.DataFrame'>
Index: 758 entries, 0 to 377
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   TeamID         758 non-null    int64  
 1   TeamName       758 non-null    object 
 2   FirstD1Season  380 non-null    float64
 3   LastD1Season   380 non-null    float64
 4   Gender         758 non-null    object 
dtypes: float64(2), int64(1), object(2)
memory usage: 35.5+ KB


# Compute Team Records



In [5]:
df_wins        = df_reg_.groupby(['WTeamID']).size().reset_index(name='Wins')
df_team_points  = df_reg_.groupby('WTeamID')["WScore"].sum()
df_opp_points  = df_reg_.groupby('WTeamID')["LScore"].sum()

df_wins = df_wins.merge( df_team_points, left_on="WTeamID", right_index=True )
df_wins = df_wins.merge( df_opp_points, left_on="WTeamID", right_index=True )

#df_wins = df_wins.merge( df_teams_, left_on="WTeamID", right_on="TeamID", how="outer")
df_wins.rename( columns={ 'WScore': 'TeamPoints', 'LScore': 'OppPoints', 'WTeamID': 'TeamID' }, inplace=True )
df_wins.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 726 entries, 0 to 725
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   TeamID      726 non-null    int64
 1   Wins        726 non-null    int64
 2   TeamPoints  726 non-null    int64
 3   OppPoints   726 non-null    int64
dtypes: int64(4)
memory usage: 22.8 KB


In [6]:
df_losses      = df_reg_.groupby(['LTeamID']).size().reset_index(name='Losses')
df_team_points = df_reg_.groupby('LTeamID')["LScore"].sum()
df_opp_points  = df_reg_.groupby('LTeamID')["WScore"].sum()

df_losses = df_losses.merge( df_team_points, left_on="LTeamID", right_index=True )
df_losses = df_losses.merge( df_opp_points,  left_on="LTeamID", right_index=True )

#df_losses = df_losses.merge( df_teams_, left_on="LTeamID", right_on="TeamID", how="outer")
df_losses.rename( columns={ 'LScore': 'TeamPoints', 'WScore': 'OppPoints', 'LTeamID': 'TeamID' }, inplace=True )
df_losses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 726 entries, 0 to 725
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   TeamID      726 non-null    int64
 1   Losses      726 non-null    int64
 2   TeamPoints  726 non-null    int64
 3   OppPoints   726 non-null    int64
dtypes: int64(4)
memory usage: 22.8 KB


In [7]:
df_summary = df_wins.merge( df_losses, on="TeamID", how='outer' )
df_summary.fillna(0, inplace=True)
df_summary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 726 entries, 0 to 725
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   TeamID        726 non-null    int64
 1   Wins          726 non-null    int64
 2   TeamPoints_x  726 non-null    int64
 3   OppPoints_x   726 non-null    int64
 4   Losses        726 non-null    int64
 5   TeamPoints_y  726 non-null    int64
 6   OppPoints_y   726 non-null    int64
dtypes: int64(7)
memory usage: 39.8 KB


## Undefeated Teams

In [8]:
df_summary[ df_summary[ "Losses" ] == 0 ]

,TeamID,Wins,TeamPoints_x,OppPoints_x,Losses,TeamPoints_y,OppPoints_y


## Shut-out Teams

In [9]:
df_summary[ df_summary[ "Wins" ] == 0 ]

,TeamID,Wins,TeamPoints_x,OppPoints_x,Losses,TeamPoints_y,OppPoints_y


In [10]:
df_summary[ "TeamPoints" ] = df_summary[ "TeamPoints_x" ] + df_summary[ "TeamPoints_y" ]
df_summary[ "OppPoints" ] = df_summary[ "OppPoints_x" ] + df_summary[ "OppPoints_y" ]
df_summary.drop( columns=[ "TeamPoints_x", "TeamPoints_y", "OppPoints_x", "OppPoints_y" ], axis=1, inplace=True )

In [11]:
df_summary["WinDiff"] = df_summary["Wins"] - df_summary["Losses"]
df_summary["PointsDiff"] = df_summary["TeamPoints"] - df_summary["OppPoints"]
df_summary["TotalGames"] = df_summary["Wins"] + df_summary["Losses"]
df_summary["WinPercentage"] = round(df_summary["Wins"] / df_summary["TotalGames"], 3)
df_summary["Season"] = YEAR

In [12]:
df_summary = df_summary.merge( df_teams_[["TeamID", "TeamName", "Gender"]], on="TeamID" )
df_summary.head(5)

,TeamID,Wins,Losses,TeamPoints,OppPoints,WinDiff,PointsDiff,TotalGames,WinPercentage,Season,TeamName,Gender
0,1101,13,16,1959,2059,-3,-100,29,0.448,2025,Abilene Chr,M
1,1102,4,28,1982,2357,-24,-375,32,0.125,2025,Air Force,M
2,1103,26,6,2687,2429,20,258,32,0.812,2025,Akron,M
3,1104,25,8,3007,2687,17,320,33,0.758,2025,Alabama,M
4,1105,7,22,2010,2317,-15,-307,29,0.241,2025,Alabama A&M,M


# Merge with Tournament Seed Data

In [13]:
df_ = load_mens_and_womens( "NCAATourneySeeds.csv" )
df_seeds = df_[ df_["Season"] == YEAR ]
df_seeds.info()

<class 'pandas.core.frame.DataFrame'>
Index: 136 entries, 2558 to 1743
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Season  136 non-null    int64 
 1   Seed    136 non-null    object
 2   TeamID  136 non-null    int64 
 3   Gender  136 non-null    object
dtypes: int64(2), object(2)
memory usage: 5.3+ KB


In [14]:
df_summary = df_summary.merge( df_seeds, on=['TeamID', 'Gender', 'Season'], how='left' )
df_summary['SeedValue'] = df_summary['Seed'].apply(extract_seed_value)
df_summary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 726 entries, 0 to 725
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   TeamID         726 non-null    int64  
 1   Wins           726 non-null    int64  
 2   Losses         726 non-null    int64  
 3   TeamPoints     726 non-null    int64  
 4   OppPoints      726 non-null    int64  
 5   WinDiff        726 non-null    int64  
 6   PointsDiff     726 non-null    int64  
 7   TotalGames     726 non-null    int64  
 8   WinPercentage  726 non-null    float64
 9   Season         726 non-null    int64  
 10  TeamName       726 non-null    object 
 11  Gender         726 non-null    object 
 12  Seed           136 non-null    object 
 13  SeedValue      120 non-null    float64
dtypes: float64(2), int64(9), object(3)
memory usage: 79.5+ KB


# Compute Team Stength of Schedule

In [15]:
teams = set(df_reg_['WTeamID'].unique()).union(df_reg_['LTeamID'].unique())
print(f"Number of teams: {len(teams)}")

Number of teams: 726


In [16]:
# Function to calculate win percentage
def win_percentage(team, df):
    wins = len(df[((df['WTeamID'] == team) & (df['WScore'] > df['LScore'])) | 
                   ((df['LTeamID'] == team) & (df['LScore'] > df['WScore']))])
    losses = len(df[((df['WTeamID'] == team) & (df['WScore'] < df['LScore'])) | 
                     ((df['LTeamID'] == team) & (df['LScore'] < df['WScore']))])
    return wins / (wins + losses) if (wins + losses) > 0 else 0

# Function to calculate OWP
def calculate_owp(team, df):
    opponents = set()
    for index, row in df.iterrows():
        if row['WTeamID'] == team:
            opponents.add(row['LTeamID'])
        elif row['LTeamID'] == team:
            opponents.add(row['WTeamID'])
    
    total_owp = 0
    for opponent in opponents:
        # Filter out games between team and opponent to avoid circularity
        temp_df = df[~((df['WTeamID'] == team) & (df['LTeamID'] == opponent) | (df['WTeamID'] == opponent) & (df['LTeamID'] == team))]
        total_owp += win_percentage(opponent, temp_df)
    return total_owp / len(opponents) if opponents else 0

# Function to calculate OOWP
def calculate_oowp(team, df):
    opponents = set()
    for index, row in df.iterrows():
        if row['WTeamID'] == team:
            opponents.add(row['LTeamID'])
        elif row['LTeamID'] == team:
            opponents.add(row['WTeamID'])
    
    total_oowp = 0
    for opponent in opponents:
        total_oowp += calculate_owp(opponent, df)
    return total_oowp / len(opponents) if opponents else 0

In [17]:
sos_data = []
for team in list(teams):
  owp = round(calculate_owp(team, df_reg_),4)
  #oowp = calculate_oowp(team, df_reg_)
  #sos = (owp * (2/3)) + (oowp * (1/3))
  sos_data.append({'TeamID': team, 'SOS': owp})

sos_df = pd.DataFrame(sos_data)
print(sos_df.sort_values(by="SOS", ascending=False))

     TeamID     SOS
247    1104  0.6602
112    3376  0.6559
263    1120  0.6545
636    3163  0.6531
653    3181  0.6528
..      ...     ...
663    3192  0.3829
725    3255  0.3807
258    1115  0.3792
724    3254  0.3710
186    3457  0.3479

[726 rows x 2 columns]


# Merge SoS with Season Results

In [18]:
df_summary = df_summary.merge( sos_df, on='TeamID', how='left' )
df_summary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 726 entries, 0 to 725
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   TeamID         726 non-null    int64  
 1   Wins           726 non-null    int64  
 2   Losses         726 non-null    int64  
 3   TeamPoints     726 non-null    int64  
 4   OppPoints      726 non-null    int64  
 5   WinDiff        726 non-null    int64  
 6   PointsDiff     726 non-null    int64  
 7   TotalGames     726 non-null    int64  
 8   WinPercentage  726 non-null    float64
 9   Season         726 non-null    int64  
 10  TeamName       726 non-null    object 
 11  Gender         726 non-null    object 
 12  Seed           136 non-null    object 
 13  SeedValue      120 non-null    float64
 14  SOS            726 non-null    float64
dtypes: float64(3), int64(9), object(3)
memory usage: 85.2+ KB


In [19]:
df_summary.sort_values(by=['WinPercentage','SOS'], ascending=False, inplace=True)
df_summary.head(10)

,TeamID,Wins,Losses,TeamPoints,OppPoints,WinDiff,PointsDiff,TotalGames,WinPercentage,Season,TeamName,Gender,Seed,SeedValue,SOS
666,3417,30,2,2516,1845,28,671,32,0.938,2025,UCLA,W,Y01,1.0,0.6252
469,3213,30,2,2427,1854,28,573,32,0.938,2025,Grand Canyon,W,Y13,13.0,0.4528
419,3163,31,3,2748,1763,28,985,34,0.912,2025,Connecticut,W,Z02,2.0,0.6531
649,3400,31,3,2684,1901,28,783,34,0.912,2025,Texas,W,X01,1.0,0.6283
74,1181,31,3,2812,2105,28,707,34,0.912,2025,Duke,M,W01,1.0,0.5746
644,3395,31,3,2638,1948,28,690,34,0.912,2025,TCU,W,X02,2.0,0.5584
626,3376,30,3,2656,1906,27,750,33,0.909,2025,South Carolina,W,W01,1.0,0.6559
674,3425,28,3,2525,1855,25,670,31,0.903,2025,USC,W,Z01,1.0,0.6052
451,3195,28,3,2207,1679,25,528,31,0.903,2025,FGCU,W,Z14,14.0,0.5211
540,3286,28,3,2226,1765,25,461,31,0.903,2025,Montana St,W,X13,13.0,0.5060


# Output to File

In [20]:
df_summary.to_csv(f"../data/kaggle/Summary.{YEAR}.csv", index=False)